<a href="https://colab.research.google.com/github/maryambahri/recession_analysis/blob/main/scripts/ingestion_WB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#installing necessary library

!pip install boto3 requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.1 MB/s eta 0:00:00


In [ ]:
# loading all libraries here and connecting to S3 bucket

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import boto3
import os
from datetime import datetime
import io

# === EDIT THESE ===
AWS_ACCESS_KEY_ID = "xxxxxxxxxx"
AWS_SECRET_ACCESS_KEY = "xxxxxxxxxxx"
AWS_REGION = "us-east-1"   # or whatever region you use
S3_BUCKET_NAME = "group24-recessionanalysis"
S3_PREFIX = "raw_data/"

# Create S3 client using your keys
s3_client = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=AWS_REGION,
)

print("S3 client created OK")


S3 client created OK


In [ ]:
# Downloading the data directly from worldbank and pushing it to S3

# IMPORTS & AWS / S3 CONFIG
import requests
import xml.etree.ElementTree as ET

# WORLD BANK CONFIG
WORLD_BANK_BASE_URL = "http://api.worldbank.org/v2"
COUNTRY_CODE = "US"

INDICATORS = {
    "GDP_USA": "NY.GDP.MKTP.CD",        # GDP (current US$)
    "GDP_GROWTH": "NY.GDP.MKTP.KD.ZG",  # Real GDP growth (annual %)
    "USA_Inflation": "FP.CPI.TOTL.ZG",  # Inflation (annual %)
    "USA_Unemployment": "SL.UEM.TOTL.ZS",  # Unemployment (% of labor force)
}

FILE_NAMES = {
    "GDP_USA": "GDP_USA.csv",
    "USA_Inflation": "USA_Inflation.csv",
    "USA_Unemployment": "USA_Unemployment.csv",
    "USA_Recession": "USA_Recession.csv",
    "USA_Unemployement_XML": "USA_Unemployement.xml",  # requested spelling
}

START_YEAR = 1960
END_YEAR = None  # or 2024 if you want to cut off

# HELPER FUNCTIONS

def fetch_world_bank_indicator(country, indicator, start_year=None, end_year=None):
    """
    Fetch a World Bank indicator for a given country and return a tidy DataFrame.
    Columns: country, indicator, year, value
    """
    params = {"format": "json", "per_page": 20000}
    if start_year:
        params["date"] = f"{start_year}:{end_year or ''}".rstrip(":")

    url = f"{WORLD_BANK_BASE_URL}/country/{country}/indicator/{indicator}"
    resp = requests.get(url, params=params)
    resp.raise_for_status()
    data = resp.json()

    # data[0] = metadata, data[1] = actual records
    records = data[1]
    rows = []
    for rec in records:
        year = rec.get("date")
        value = rec.get("value")
        if year is None:
            continue
        try:
            year_int = int(year)
        except ValueError:
            continue
        if start_year and year_int < start_year:
            continue
        if end_year and year_int > end_year:
            continue
        rows.append(
            {
                "country": rec.get("country", {}).get("value"),
                "indicator": indicator,
                "year": year_int,
                "value": value,
            }
        )

    df = pd.DataFrame(rows).sort_values("year").reset_index(drop=True)
    return df


def create_recession_table(gdp_growth_df: pd.DataFrame) -> pd.DataFrame:
    """
    Simple recession flag: year is recession if real GDP growth < 0.
    """
    df = gdp_growth_df.copy()
    df.rename(columns={"value": "gdp_growth_percent"}, inplace=True)
    df["is_recession"] = df["gdp_growth_percent"].apply(
        lambda x: 1 if (x is not None and pd.notna(x) and x < 0) else 0
    )
    return df[["country", "year", "gdp_growth_percent", "is_recession"]].sort_values("year")


def build_unemployment_xml(unemp_df: pd.DataFrame) -> bytes:
    """
    Build XML content for USA unemployment data and return as bytes.

    Example structure:
    <USA_Unemployment country="USA">
        <Year value="1990" unemployment_percent="5.2" />
        ...
    </USA_Unemployment>
    """
    root = ET.Element("USA_Unemployment", country="USA")

    for _, row in unemp_df.iterrows():
        year = row["year"]
        value = row["value"]

        year_el = ET.SubElement(root, "Year")
        year_el.set("value", str(year))
        if pd.notna(value):
            year_el.set("unemployment_percent", str(value))

    xml_bytes = ET.tostring(root, encoding="utf-8", xml_declaration=True)
    return xml_bytes


def upload_bytes_to_s3(data_bytes: bytes, bucket: str, key: str, content_type: str):
    """
    Upload raw bytes directly to S3.
    """
    print(f"Uploading to s3://{bucket}/{key} ...")
    s3_client.put_object(
        Bucket=bucket,
        Key=key,
        Body=data_bytes,
        ContentType=content_type,
    )
    print("Upload done.")


# Fetch data from World Bank
gdp_df = fetch_world_bank_indicator(COUNTRY_CODE, INDICATORS["GDP_USA"], START_YEAR, END_YEAR)
gdp_growth_df = fetch_world_bank_indicator(COUNTRY_CODE, INDICATORS["GDP_GROWTH"], START_YEAR, END_YEAR)
infl_df = fetch_world_bank_indicator(COUNTRY_CODE, INDICATORS["USA_Inflation"], START_YEAR, END_YEAR)
unemp_df = fetch_world_bank_indicator(COUNTRY_CODE, INDICATORS["USA_Unemployment"], START_YEAR, END_YEAR)

# Create recession table
recession_df = create_recession_table(gdp_growth_df)

# Convert DataFrames to CSV IN MEMORY and upload

# GDP CSV
gdp_csv_buf = io.StringIO()
gdp_df[["year", "value"]].rename(columns={"value": "gdp_current_usd"}).to_csv(gdp_csv_buf, index=False)
gdp_bytes = gdp_csv_buf.getvalue().encode("utf-8")
upload_bytes_to_s3(
    gdp_bytes,
    S3_BUCKET_NAME,
    f"{S3_PREFIX}{FILE_NAMES['GDP_USA']}",
    content_type="text/csv",
)

# Inflation CSV
infl_csv_buf = io.StringIO()
infl_df[["year", "value"]].rename(columns={"value": "inflation_annual_percent"}).to_csv(infl_csv_buf, index=False)
infl_bytes = infl_csv_buf.getvalue().encode("utf-8")
upload_bytes_to_s3(
    infl_bytes,
    S3_BUCKET_NAME,
    f"{S3_PREFIX}{FILE_NAMES['USA_Inflation']}",
    content_type="text/csv",
)

# Unemployment CSV
unemp_csv_buf = io.StringIO()
unemp_df[["year", "value"]].rename(columns={"value": "unemployment_percent"}).to_csv(unemp_csv_buf, index=False)
unemp_bytes = unemp_csv_buf.getvalue().encode("utf-8")
upload_bytes_to_s3(
    unemp_bytes,
    S3_BUCKET_NAME,
    f"{S3_PREFIX}{FILE_NAMES['USA_Unemployment']}",
    content_type="text/csv",
)

# Recession CSV
recession_csv_buf = io.StringIO()
recession_df.to_csv(recession_csv_buf, index=False)
recession_bytes = recession_csv_buf.getvalue().encode("utf-8")
upload_bytes_to_s3(
    recession_bytes,
    S3_BUCKET_NAME,
    f"{S3_PREFIX}{FILE_NAMES['USA_Recession']}",
    content_type="text/csv",
)

# Build unemployment XML IN MEMORY and upload
xml_bytes = build_unemployment_xml(unemp_df)
upload_bytes_to_s3(
    xml_bytes,
    S3_BUCKET_NAME,
    f"{S3_PREFIX}{FILE_NAMES['USA_Unemployement_XML']}",
    content_type="application/xml",
)
print("All in-memory uploads completed to S3.")


Uploading to s3://group24-recessionanalysis/cleansed_data/GDP_USA.csv ...
Upload done.
Uploading to s3://group24-recessionanalysis/cleansed_data/USA_Inflation.csv ...
Upload done.
Uploading to s3://group24-recessionanalysis/cleansed_data/USA_Unemployment.csv ...
Upload done.
Uploading to s3://group24-recessionanalysis/cleansed_data/USA_Recession.csv ...
Upload done.
Uploading to s3://group24-recessionanalysis/cleansed_data/USA_Unemployement.xml ...
Upload done.
All in-memory uploads completed to S3.


# **Cleansing Raw GDP file**

data source: WorldBank
retrieved 11/16/2025

In [ ]:
import pandas as pd
from pandas.errors import ParserError

paginator = s3_client.get_paginator("list_objects_v2")

dataframes = {}

for page in paginator.paginate(Bucket=S3_BUCKET_NAME, Prefix=S3_PREFIX):
    for obj in page.get("Contents", []):
        key = obj["Key"]

        if key.endswith("/"):
            continue

        print(f"Found file in S3: {key}")

        if key.lower().endswith(".csv"):
            try:
                # First attempt: plain CSV
                s3_obj = s3_client.get_object(Bucket=S3_BUCKET_NAME, Key=key)
                df = pd.read_csv(s3_obj["Body"])
            except ParserError:
                # Second attempt: World Bank-style CSV with 4 metadata lines
                print(f"ParserError for {key}, retrying with skiprows=4")
                s3_obj = s3_client.get_object(Bucket=S3_BUCKET_NAME, Key=key)
                df = pd.read_csv(s3_obj["Body"], skiprows=4)
        elif key.lower().endswith((".xlsx", ".xls")):
            s3_obj = s3_client.get_object(Bucket=S3_BUCKET_NAME, Key=key)
            df = pd.read_excel(s3_obj["Body"])
        else:
            print(f"Skipping unsupported file type: {key}")
            continue

        base_name = os.path.basename(key)
        name_no_ext = os.path.splitext(base_name)[0]
        dataframes[name_no_ext] = df
        print(f"Loaded into DataFrame: dataframes['{name_no_ext}']")

print("\nLoaded DataFrames:")
for name, df in dataframes.items():
    print(f"- {name}: {df.shape}")


Found file in S3: raw_data/GDP_USA.csv
ParserError for raw_data/GDP_USA.csv, retrying with skiprows=4
Loaded into DataFrame: dataframes['GDP_USA']
Found file in S3: raw_data/USA_Inflation.csv
ParserError for raw_data/USA_Inflation.csv, retrying with skiprows=4
Loaded into DataFrame: dataframes['USA_Inflation']
Found file in S3: raw_data/USA_Recession.csv
Loaded into DataFrame: dataframes['USA_Recession']
Found file in S3: raw_data/USA_Unemployement.xml
Skipping unsupported file type: raw_data/USA_Unemployement.xml
Found file in S3: raw_data/USA_Unemployment.csv
ParserError for raw_data/USA_Unemployment.csv, retrying with skiprows=4
Loaded into DataFrame: dataframes['USA_Unemployment']
Found file in S3: raw_data/USA_Unemployment.xml
Skipping unsupported file type: raw_data/USA_Unemployment.xml

Loaded DataFrames:
- GDP_USA: (266, 70)
- USA_Inflation: (266, 70)
- USA_Recession: (230, 2)
- USA_Unemployment: (266, 70)


In [ ]:
# importing all libraries here

import numpy as np
import pandas as pd

!git clone https://github.com/maryambahri/recession_analysis.git


Cloning into 'recession_analysis'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 75 (delta 23), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (75/75), 249.11 KiB | 4.70 MiB/s, done.
Resolving deltas: 100% (23/23), done.


In [ ]:
# Get the in-memory DataFrame
raw_df = dataframes["GDP_USA"].copy()

# filter only United States rows:
raw_df = raw_df[raw_df["Country Name"] == "United States"]

# Look at the first few rows
raw_df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
251,United States,USA,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,2.565343,6.129637,4.357286,5.762747,6.498454,...,1.819451,2.457622,2.966505,2.583825,-2.163029,6.055053,2.512375,2.887556,2.79619,NaN


In [ ]:
# we neat things up here and transpose years

# list of year columns
year_cols = [c for c in raw_df.columns if c.isdigit()]

usa_gdp = raw_df.melt(
    id_vars=["Country Name", "Country Code", "Indicator Name", "Indicator Code"],
    value_vars=year_cols,
    var_name="Year",
    value_name="GDP_value"
)

# Cleaning up
usa_gdp["Year"] = usa_gdp["Year"].astype(int)
usa_gdp = usa_gdp.sort_values("Year").reset_index(drop=True)
usa_gdp = usa_gdp.dropna(subset=["GDP_value"])  # drop missing years if you want

usa_gdp.head()



,Country Name,Country Code,Indicator Name,Indicator Code,Year,GDP_value
1,United States,USA,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,1961,2.565343
2,United States,USA,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,1962,6.129637
3,United States,USA,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,1963,4.357286
4,United States,USA,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,1964,5.762747
5,United States,USA,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,1965,6.498454


# **Cleansing Raw Unemployment file**

data source: WorldBank
retrieved 11/16/2025

In [ ]:
# Get the in-memory DataFrame
raw_df = dataframes["USA_Unemployment"].copy()

# filter only United States rows:
raw_df = raw_df[raw_df["Country Name"] == "United States"]

# Look at the first few rows
raw_df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
251,United States,USA,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.NE.ZS,5.5,6.7,5.5,5.7,5.2,4.5,...,4.869,4.355,3.896,3.669,8.055,5.349,3.65,3.638,4.022,NaN


In [ ]:
# we neat things up here and transpose years

# list of year columns
year_cols = [c for c in raw_df.columns if c.isdigit()]

usa_unemployment = raw_df.melt(
    id_vars=["Country Name", "Country Code", "Indicator Name", "Indicator Code"],
    value_vars=year_cols,
    var_name="Year",
    value_name="Unemployment_value"
)

# cleaning up
usa_unemployment["Year"] = usa_unemployment["Year"].astype(int)
usa_unemployment = usa_unemployment.sort_values("Year").reset_index(drop=True)
usa_unemployment = usa_unemployment.dropna(subset=["Unemployment_value"])  # drop missing years if you want

usa_unemployment.head()


,Country Name,Country Code,Indicator Name,Indicator Code,Year,Unemployment_value
0,United States,USA,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.NE.ZS,1960,5.5
1,United States,USA,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.NE.ZS,1961,6.7
2,United States,USA,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.NE.ZS,1962,5.5
3,United States,USA,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.NE.ZS,1963,5.7
4,United States,USA,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.NE.ZS,1964,5.2


# **Cleansing Raw Inflation file**

data source: WorldBank
retrieved 11/16/2025

In [ ]:
# Get the in-memory DataFrame
raw_df = dataframes["USA_Inflation"].copy()

# filter only United States rows:
raw_df = raw_df[raw_df["Country Name"] == "United States"]

# Look at the first few rows
raw_df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
251,United States,USA,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,1.457976,1.070724,1.198773,1.239669,1.278912,1.585169,...,1.261583,2.13011,2.442583,1.81221,1.233584,4.697859,8.0028,4.116338,2.949525,NaN


In [ ]:
# we neat things up here and transpose years

# list of year columns
year_cols = [c for c in raw_df.columns if c.isdigit()]

usa_inflation = raw_df.melt(
    id_vars=["Country Name", "Country Code", "Indicator Name", "Indicator Code"],
    value_vars=year_cols,
    var_name="Year",
    value_name="Inflation_value"
)

# cleaning up
usa_inflation["Year"] = usa_inflation["Year"].astype(int)
usa_inflation = usa_inflation.sort_values("Year").reset_index(drop=True)
usa_inflation = usa_inflation.dropna(subset=["Inflation_value"])  # drop missing years if you want

usa_inflation.head()

,Country Name,Country Code,Indicator Name,Indicator Code,Year,Inflation_value
0,United States,USA,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,1960,1.457976
1,United States,USA,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,1961,1.070724
2,United States,USA,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,1962,1.198773
3,United States,USA,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,1963,1.239669
4,United States,USA,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,1964,1.278912


# **Cleansing Raw Recession file**

data source: WorldBank
retrieved 11/16/2025

In [ ]:
# Get the in-memory DataFrame
raw_df = dataframes["USA_Recession"].copy()

# Look at the first few rows
raw_df.head()

,observation_date,JHDUSRGDPBR
0,1967-10-01,0
1,1968-01-01,0
2,1968-04-01,0
3,1968-07-01,0
4,1968-10-01,0


In [ ]:
# Rename columns
usa_recession = raw_df.rename(columns={
    "observation_date": "Year",
    "JHDUSRGDPBR": "Recession_Tag"
})

# Convert date to year only
usa_recession["Year"] = pd.to_datetime(usa_recession["Year"]).dt.year

usa_recession.head()


,Year,Recession_Tag
0,1967,0
1,1968,0
2,1968,0
3,1968,0
4,1968,0


----------------------------------------------------------------

# **Preprocessing Data**

In [ ]:
# alighning data start years

usa_GDP_startYear=usa_gdp["Year"].min()
usa_GDP_endYear=usa_gdp["Year"].max()

usa_unemployment_startYear=usa_unemployment["Year"].min()
usa_unemployment_endYear=usa_unemployment["Year"].max()

usa_inflation_startYear=usa_inflation["Year"].min()
usa_inflation_endYear=usa_inflation["Year"].max()

usa_recession_startYear=usa_recession["Year"].min()
usa_recession_endYear=usa_recession["Year"].max()

print("USA GDP start year: ",usa_GDP_startYear)
print("USA GDP end year: ",usa_GDP_endYear)
print("USA Unemployment start year: ",usa_unemployment_startYear)
print("USA Unemployment end year: ",usa_unemployment_endYear)
print("USA Inflation start year: ",usa_inflation_startYear)
print("USA Inflation end year: ",usa_inflation_endYear)
print("USA Recession start year: ",usa_recession_startYear)
print("USA Recession end year: ",usa_recession_endYear)


USA GDP start year:  1961
USA GDP end year:  2024
USA Unemployment start year:  1960
USA Unemployment end year:  2024
USA Inflation start year:  1960
USA Inflation end year:  2024
USA Recession start year:  1967
USA Recession end year:  2025


In [ ]:
# instead of hardcoding year alighnments, we will do it dynamically as data refresh

common_start = max(usa_GDP_startYear,
               usa_unemployment_startYear,
               usa_inflation_startYear,
               usa_recession_startYear)

common_end = min(usa_GDP_endYear,
               usa_unemployment_endYear,
               usa_inflation_endYear,
               usa_recession_endYear)



usa_gdp = usa_gdp[(usa_gdp["Year"] >= common_start) & (usa_gdp["Year"] <= common_end)]
usa_unemployment = usa_unemployment[(usa_unemployment["Year"] >= common_start) & (usa_unemployment["Year"] <= common_end)]
usa_inflation = usa_inflation[(usa_inflation["Year"] >= common_start) & (usa_inflation["Year"] <= common_end)]
usa_recession = usa_recession[(usa_recession["Year"] >= common_start) & (usa_recession["Year"] <= common_end)]


In [ ]:
# recheck alighnment
usa_GDP_startYear=usa_gdp["Year"].min()
usa_GDP_endYear=usa_gdp["Year"].max()

usa_unemployment_startYear=usa_unemployment["Year"].min()
usa_unemployment_endYear=usa_unemployment["Year"].max()

usa_inflation_startYear=usa_inflation["Year"].min()
usa_inflation_endYear=usa_inflation["Year"].max()

usa_recession_startYear=usa_recession["Year"].min()
usa_recession_endYear=usa_recession["Year"].max()

print("USA GDP start year: ",usa_GDP_startYear)
print("USA GDP end year: ",usa_GDP_endYear)
print("USA Unemployment start year: ",usa_unemployment_startYear)
print("USA Unemployment end year: ",usa_unemployment_endYear)
print("USA Inflation start year: ",usa_inflation_startYear)
print("USA Inflation end year: ",usa_inflation_endYear)
print("USA Recession start year: ",usa_recession_startYear)
print("USA Recession end year: ",usa_recession_endYear)


USA GDP start year:  1967
USA GDP end year:  2024
USA Unemployment start year:  1967
USA Unemployment end year:  2024
USA Inflation start year:  1967
USA Inflation end year:  2024
USA Recession start year:  1967
USA Recession end year:  2024


In [ ]:
# Brief EDA

usa_gdp.describe()

,Year,GDP_value
count,58.000000,58.000000
mean,1995.500000,2.790586
std,16.886879,1.998545
min,1967.000000,-2.576500
25%,1981.250000,2.032351
50%,1995.500000,2.841873
75%,2009.750000,4.065445
max,2024.000000,7.236453


In [ ]:
usa_unemployment.describe()

,Year,Unemployment_value
count,58.000000,58.000000
mean,1995.500000,5.962379
std,16.886879,1.674345
min,1967.000000,3.500000
25%,1981.250000,4.765500
50%,1995.500000,5.625000
75%,2009.750000,7.100000
max,2024.000000,9.700000


In [ ]:
usa_inflation.describe()

,Year,Inflation_value
count,58.000000,58.000000
mean,1995.500000,4.024215
std,16.886879,2.780287
min,1967.000000,-0.355546
25%,1981.250000,2.286994
50%,1995.500000,3.219190
75%,2009.750000,4.794717
max,2024.000000,13.549202


In [ ]:
usa_recession.describe()

,Year,Recession_Tag
count,229.000000,229.000000
mean,1995.873362,0.179039
std,16.563187,0.384225
min,1967.000000,0.000000
25%,1982.000000,0.000000
50%,1996.000000,0.000000
75%,2010.000000,0.000000
max,2024.000000,1.000000


In [ ]:
# checking for missing values
usa_gdp.isna().sum()

,0
Country Name,0
Country Code,0
Indicator Name,0
Indicator Code,0
Year,0
GDP_value,0


In [ ]:
usa_unemployment.isna().sum()

,0
Country Name,0
Country Code,0
Indicator Name,0
Indicator Code,0
Year,0
Unemployment_value,0


In [ ]:
usa_inflation.isna().sum()

,0
Country Name,0
Country Code,0
Indicator Name,0
Indicator Code,0
Year,0
Inflation_value,0


In [ ]:
usa_recession.isna().sum()

,0
Year,0
Recession_Tag,0


In [ ]:
# Rename columns for clear presentation
usa_gdp = usa_gdp.rename(columns={"GDP_value": "GDP_growth_rate"})
usa_unemployment = usa_unemployment.rename(columns={"Unemployment_value": "Unemployment_rate"})
usa_inflation = usa_inflation.rename(columns={"Inflation_value": "Inflation_rate"})

# Keep only needed columns
usa_gdp = usa_gdp[["Year", "GDP_growth_rate"]]
usa_unemployment = usa_unemployment[["Year", "Unemployment_rate"]]
usa_inflation = usa_inflation[["Year", "Inflation_rate"]]
usa_recession = usa_recession[["Year", "Recession_Tag"]]

# Merge step-by-step on "Year"
merged_df = usa_gdp.merge(usa_unemployment, on="Year", how="inner")
merged_df = merged_df.merge(usa_inflation, on="Year", how="inner")
merged_df = merged_df.merge(usa_recession, on="Year", how="inner")

# Sort and clean final dataset
merged_df = merged_df.sort_values("Year").reset_index(drop=True)

merged_df.head()


,Year,GDP_growth_rate,Unemployment_rate,Inflation_rate,Recession_Tag
0,1967,2.742666,3.8,2.772786,0
1,1968,4.914509,3.6,4.271796,0
2,1968,4.914509,3.6,4.271796,0
3,1968,4.914509,3.6,4.271796,0
4,1968,4.914509,3.6,4.271796,0


In [ ]:
S3_BUCKET_NAME = "group24-recessionanalysis"
S3_PREFIX = "cleansed_data/"

# build date string, e.g. 20251123
date_str = datetime.now().strftime("%Y%m%d")

# filename with date
file_name = f"processed_dataset_{date_str}.csv"
s3_key = f"{S3_PREFIX}{file_name}"

# convert merged_df to CSV in memory
csv_buffer = io.StringIO()
merged_df.to_csv(csv_buffer, index=False)

# upload to S3
s3_client.put_object(
    Bucket=S3_BUCKET_NAME,
    Key=s3_key,
    Body=csv_buffer.getvalue(),
    ContentType="text/csv",
)

print(f"Uploaded cleaned file to s3://{S3_BUCKET_NAME}/{s3_key}")

Uploaded cleaned file to s3://group24-recessionanalysis/cleansed_data/processed_dataset_20251123.csv


In [ ]:
# save cleansed table locally

from google.colab import files
import pandas as pd

# Create the CSV in Colab
output_file = "processed_dataset.csv"
merged_df.to_csv(output_file, index=False)

# Download to local machine
files.download(output_file)



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

--------------------------------------------------------------------------------